In [1]:
import pyiast
import pygaps
import pygaps.characterisation as pgc
import pygaps.parsing as pgp
import pygaps.modelling as pgm
import pygaps.graphing as pgg
from pygaps import iast
import pandas as pd
import numpy as np
from datetime import datetime
import os, glob
import pathlib
%matplotlib qt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

298 K 

CO2/N2 (400ppm, 1%, 4%, 15%)

CO2/CH4 (1/99, 5/95, 50/50)

In [2]:
df = pd.read_csv('WO2F4_IAST.csv')

CO2_298K = df[['CO2_x','CO2_y']]
CO2_298K.columns = ['Pressure(bar)','Loading(mmol/g)']
N2_298K = df[['N2_x','N2_y']]
N2_298K.columns = ['Pressure(bar)','Loading(mmol/g)']
CH4_298K = df[['CH4_x','CH4_y']]
CH4_298K.columns = ['Pressure(bar)','Loading(mmol/g)']

df.head()

,CO2_x,CO2_y,N2_x,N2_y,CH4_x,CH4_y
0,0.000095,0.362149,0.010438,0.000607,0.010125,0.000857
1,0.000189,0.657772,0.013310,0.000915,0.013362,0.001499
2,0.000496,0.934412,0.023167,0.002498,0.019811,0.002851
3,0.000695,1.075123,0.026699,0.002904,0.026665,0.004145
4,0.001008,1.204073,0.032961,0.003676,0.033150,0.005653


In [3]:
WO2F4_CO2 = pygaps.PointIsotherm(
    pressure=CO2_298K['Pressure(bar)'].dropna().tolist(),  
    loading=CO2_298K['Loading(mmol/g)'].dropna().tolist(),
    m='WO2F4-1-Ni',a='carbon dioxide',t='298',
    pressure_mode = 'absolute',pressure_unit = 'bar',material_basis = 'mass',
    material_unit = 'g',loading_basis = 'molar',loading_unit = 'mmol',
    temperature_unit = 'K'
)

WO2F4_N2 = pygaps.PointIsotherm(
    pressure=N2_298K['Pressure(bar)'].dropna().tolist(),  
    loading=N2_298K['Loading(mmol/g)'].dropna().tolist(),
    m='WO2F4-1-Ni',a='nitrogen',t='298',
    pressure_mode = 'absolute',pressure_unit = 'bar',material_basis = 'mass',
    material_unit = 'g',loading_basis = 'molar',loading_unit = 'mmol',
    temperature_unit = 'K'
)

WO2F4_CH4 = pygaps.PointIsotherm(
    pressure=CH4_298K['Pressure(bar)'].dropna().tolist(),  
    loading=CH4_298K['Loading(mmol/g)'].dropna().tolist(),
    m='WO2F4-1-Ni',a='methane',t='298',
    pressure_mode = 'absolute',pressure_unit = 'bar',material_basis = 'mass',
    material_unit = 'g',loading_basis = 'molar',loading_unit = 'mmol',
    temperature_unit = 'K'
)

In [4]:
co2_model = pgm.model_iso(WO2F4_CO2, model='DSLangmuir',verbose=True)
print(co2_model.model)

n2_model = pgm.model_iso(WO2F4_N2, model='Henry',verbose=True)
print(n2_model.model)

ch4_model = pgm.model_iso(WO2F4_CH4, model='Henry',verbose=True)
print(ch4_model.model)

Attempting to model using DSLangmuir.
Model DSLangmuir success, RMSE is 0.0184
DSLangmuir isotherm model.
RMSE = 0.01844
Model parameters:
	n_m1 = 0.1595
	K1 = 1.603
	n_m2 = 1.66
	K2 = 2840
Model applicable range:
	Pressure range: 9.45e-05 - 1
	Loading range: 0.362 - 1.76

Attempting to model using Henry.
Model Henry success, RMSE is 0.00192
Henry isotherm model.
RMSE = 0.001917
Model parameters:
	K = 0.1288
Model applicable range:
	Pressure range: 0.0104 - 1
	Loading range: 0.000607 - 0.124

Attempting to model using Henry.
Model Henry success, RMSE is 0.00291
Henry isotherm model.
RMSE = 0.002906
Model parameters:
	K = 0.1917
Model applicable range:
	Pressure range: 0.0101 - 1
	Loading range: 0.000857 - 0.185



In [5]:
DAC_conc = 400/1000000
ROOM_conc = 0.01
NGCC_conc = 0.04
FLUE_conc = 0.15

CO2_concs = [DAC_conc, ROOM_conc, NGCC_conc, FLUE_conc]

for i in CO2_concs:
    results = pygaps.iast.pgiast.iast_binary_svp(isotherms=[co2_model,n2_model],
                                                 mole_fractions=[i,1-i],
                                                 pressures=list(np.linspace(0.1,1,100)),
                                                 warningoff=True,
                                                 verbose=True)
    globals()['df_'+str(i)] = pd.DataFrame(results)
    txt = globals()['df_'+str(i)]['selectivity'].iloc[-1]
    print('for a concentration of '+str(i*100)+'% CO2 in N2')
    print('CO2/N2 selectivity = '+f'{txt:.2f}'+' at 1 bar total pressure.')

for a concentration of 0.04% CO2 in N2
CO2/N2 selectivity = 23711.40 at 1 bar total pressure.
for a concentration of 1.0% CO2 in N2
CO2/N2 selectivity = 4288.31 at 1 bar total pressure.
for a concentration of 4.0% CO2 in N2
CO2/N2 selectivity = 1510.47 at 1 bar total pressure.
for a concentration of 15.0% CO2 in N2
CO2/N2 selectivity = 517.39 at 1 bar total pressure.


In [6]:
CO2_1_99 = 0.01
CO2_5_95 = 0.05
CO2_1_1 = 0.5

CO2_concs = [CO2_1_99,CO2_5_95,CO2_1_1]

for i in CO2_concs:
    results = pygaps.iast.pgiast.iast_binary_svp(isotherms=[co2_model,ch4_model],
                                                 mole_fractions=[i,1-i],
                                                 pressures=list(np.linspace(0.1,1,100)),
                                                 warningoff=True,
                                                 verbose=True)
    globals()['df_'+str(i)] = pd.DataFrame(results)
    txt = globals()['df_'+str(i)]['selectivity'].iloc[-1]
    print('for a concentration of '+str(i*100)+'% CO2 in CH4')
    print('CO2/CH4 selectivity = '+f'{txt:.2f}'+' at 1 bar total pressure.')

for a concentration of 1.0% CO2 in CH4
CO2/CH4 selectivity = 2858.93 at 1 bar total pressure.
for a concentration of 5.0% CO2 in CH4
CO2/CH4 selectivity = 845.72 at 1 bar total pressure.
for a concentration of 50.0% CO2 in CH4
CO2/CH4 selectivity = 125.85 at 1 bar total pressure.
